In [8]:
# Import external tools:
import time 
import random
from matplotlib import pyplot as plt
import librosa
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
from torchsummary import summary
import torchaudio
import soundfile as sf
from scipy import signal
import getpass
import pandas as pd
import numpy as np
import sys
import os
import importlib
from scipy.io import wavfile
from IPython.display import Audio
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# I am running this script on two different computers, so i need to change paths
# depending on computer in use: 
if getpass.getuser()=="joanna.luberadzka":
    projectdir="/Users/joanna.luberadzka/Documents/VAE-IR/"
    datadir="/Users/joanna.luberadzka/Documents/Data/IR_Arni_upload_numClosed_0-5/"
elif getpass.getuser()=="ubuntu":
    projectdir="/home/ubuntu/joanna/VAE-IR/"
    datadir="/home/ubuntu/Data/IR_Arni_upload_numClosed_0-5/"

# Add path of this project
sys.path.insert(0, projectdir+'src/')

# Import and automatically reload my own modules:
import models; importlib.reload(models)
import train; importlib.reload(train)
import datasetprep as dsprep; importlib.reload(dsprep)
import helpers; importlib.reload(helpers)

<module 'helpers' from '/home/ubuntu/joanna/VAE-IR/src/helpers.py'>

In [10]:

# load trained model:
model=models.VAE_Lin(x_len=24000).to("cpu")
model.load_state_dict(torch.load(projectdir + "models/trained_model_01-02-2023--12-17.pth")).to("cpu")
# put the model in evaluation mode
model.eval()

# data:
if getpass.getuser()=="joanna.luberadzka":
    INFO_FILE = projectdir + "irstats_ARNIandBUT_datura.csv"
elif getpass.getuser()=="ubuntu":
    INFO_FILE = projectdir+"irstats_ARNIandBUT_datura.csv"

SAMPLING_RATE=8e3
# instantiate data set 
dataset = dsprep.DatasetRirs(INFO_FILE,SAMPLING_RATE)


# *** Encoding: Using a trained variational autoencoder model, 
# generate a lower-dimensional embedding for each impulse response.

embeddings_mu=[] # list for storing ir embeddings
embeddings_rt=[] # list for storing rt values
embeddings_drr=[] # list for storing drr values
embeddings_isarni=[] # list for storing bool indicating which database
embeddings_edt=[] # list for storing edt vallues
embeddings_cte=[] # list for storing cte values

# take 100 random irs from the data set
ir_rand_indices=random.sample(range(len(dataset)),100)

for i in ir_rand_indices:
    # get info of an impulse response with a specific index
    ir, labels= dataset[i]
    # encode the input into mu and sigma (standard in VAE)
    mu, sigma = model.encode(ir)
    # mu is the embedding (sigma provides additional info about the uncertainty of this embedding)   
    emb = mu.squeeze(dim=0) 
    # convert to numpy array and append the list of embeddings
    embeddings_mu.append(emb.detach().cpu().numpy())
    embeddings_rt.append(labels["rt"])
    embeddings_drr.append(labels["drr"])
    embeddings_edt.append(labels["edt"])
    embeddings_cte.append(labels["cte"])
    embeddings_isarni.append(labels["isarni"])

# covert from list of arrays to one array
embeddings_mu=np.array(embeddings_mu)



RuntimeError: CUDA out of memory. Tried to allocate 1.07 GiB (GPU 0; 16.00 GiB total capacity; 0 bytes already allocated; 402.05 MiB free; 0 bytes reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [3]:
print(embeddings_mu[1])
print(embeddings_mu[2])
print(embeddings_mu[30])

[ 8.4131025e-06 -4.6789646e-06  2.4437904e-06  3.3527613e-06
  8.1956387e-07 -1.8328428e-06  1.4603138e-06 -3.2335520e-06
 -2.3841858e-06  2.2202730e-06  2.0861626e-06  4.2319298e-06
 -1.0952353e-06  2.5928020e-06  1.1473894e-06 -1.0728836e-06
 -7.5623393e-07 -1.1920929e-06  4.9769878e-06 -4.8428774e-06
  1.0930002e-05 -1.3411045e-07 -4.3213367e-07 -5.6177378e-06]
[ 8.4131025e-06 -4.6789646e-06  2.4437904e-06  3.3527613e-06
  8.1956387e-07 -1.8328428e-06  1.4603138e-06 -3.2335520e-06
 -2.3841858e-06  2.2202730e-06  2.0861626e-06  4.2319298e-06
 -1.0952353e-06  2.5928020e-06  1.1473894e-06 -1.0728836e-06
 -7.5623393e-07 -1.1920929e-06  4.9769878e-06 -4.8428774e-06
  1.0930002e-05 -1.3411045e-07 -4.3213367e-07 -5.6177378e-06]
[ 8.4131025e-06 -4.6789646e-06  2.4437904e-06  3.3527613e-06
  8.1956387e-07 -1.8328428e-06  1.4603138e-06 -3.2335520e-06
 -2.3841858e-06  2.2202730e-06  2.0861626e-06  4.2319298e-06
 -1.0952353e-06  2.5928020e-06  1.1473894e-06 -1.0728836e-06
 -7.5623393e-07 -1.192

In [3]:

# *** Visualization: To visualize each encoding, the D-dimensional embeddings 
# have to be reduced to 2 dimensions. This can be done with two methods: 
# PCA (linear) or TSNE (non-linear)
embeddings_pca=PCA(n_components=2).fit_transform(embeddings_mu)
embeddings_tsne = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=30).fit_transform(embeddings_mu)

plt.figure()
plt.subplot(1,2,1)
plt.scatter(embeddings_pca[:,0] , embeddings_pca[:,1])
plt.subplot(1,2,2)
plt.scatter(embeddings_pca[:,0] , embeddings_pca[:,1])

plt.figure(figsize=(10,6))
plt.subplot (2,3,1)
plt.scatter(embeddings_tsne[:,0] , embeddings_tsne[:,1], c=embeddings_rt)
plt.title('color: rt')
plt.subplot (2,3,2)
plt.scatter(embeddings_tsne[:,0] , embeddings_tsne[:,1], c=embeddings_drr)
plt.title('color: drr')
plt.subplot (2,3,3)
plt.scatter(embeddings_tsne[:,0] , embeddings_tsne[:,1], c=embeddings_edt)
plt.title('color: edt')
plt.subplot (2,3,4)
plt.scatter(embeddings_tsne[:,0] , embeddings_tsne[:,1], c=embeddings_cte)
plt.title('color: cte')
plt.subplot (2,3,5)
plt.scatter(embeddings_tsne[:,0] , embeddings_tsne[:,1], c=embeddings_isarni)
plt.title('color: isarni')



# # use pca init for repeatable results
# pca      = PCA(n_components=2)
# x_reduce = pca.fit_transform(z_df)


# emb_tsne = TSNE(n_components=2, learning_rate='auto').fit_transform(embeddings.detach().cpu().numpy())
# plt.scatter(emb_tsne[:,0] , emb_tsne[:,1], c=category_mappings)
# plt.show()


# pick a random IRs for each IR choose the closest and the furthest embedding from the set of embeddings. 

# get info of an impulse response with a specific index
# print("filename= "+ IrData["filepath"][2])
# print("rt= "+ str(IrData["rt"][2]))
# print("drr= "+ str(IrData["drr"][2]))

# convolve signal with a closest and furthest ir - the one with closest should sound similar and the one with furthest should sound different

# y = signal.fftconvolve(x, h, mode = 'full')
# sf.write("convolved.wav", y.T, sr_h, subtype='PCM_24')
# sf.write("anechoic.wav", x.T, sr_h, subtype='PCM_24')



# # plot parameters of impulse responses in the data  
# fig, ax = plt.subplots()
# ax.scatter(x=IrData["rt"][0:11331], y=IrData["drr"][0:11331],label="Arni", alpha=0.2)
# ax.scatter(x=IrData["rt"][11331:], y=IrData["drr"][11331:],label="BUT",alpha=0.2)
# plt.xlabel("T60")
# plt.ylabel("DRR")
# plt.legend()
# ax.set(title="Two IR data bases")
# plt.show()

# fig, ax = plt.subplots()
# ax.scatter(x=IrData["rt"], y=IrData["drr"],c=IrData["cte"], alpha=0.2)
# sc1=plt.xlabel("T60")
# sc2=plt.ylabel("DRR")
# ax.set(title="Early to late reflections (CTE) ")
# plt.show()

# fig, ax = plt.subplots()
# ax.scatter(x=IrData["rt"], y=IrData["drr"],c=IrData["edt"], alpha=0.2)
# plt.xlabel("T60")
# plt.ylabel("DRR")
# ax.set(title="Early decay time (EDT)")
# plt.show()

NameError: name 'embeddings_mu' is not defined

In [ ]:
# get info of an impulse response with a specific index
filename= IrData["filepath"][1200]
# load chosen RIR
h, sr_h= torchaudio.load(filename)
# load anechoic sound file
x, sr_x=torchaudio.load("anechoic.wav")
# resample anechoic signal
x=torchaudio.transforms.Resample(sr_x,sr_h)(x)
# convolve anechoic signal with RIR: y=x*h
y = signal.fftconvolve(x, h, mode = 'full')
# set signal levels
x=x.numpy()
x=helpers.set_level(x,-30)
y=helpers.set_level(y,-30)
# save signals
sf.write("convolved.wav", y.T, sr_h, subtype='PCM_24')
sf.write("anechoic.wav", x.T, sr_h, subtype='PCM_24')

In [ ]:
Audio('convolved.wav')

In [ ]:
Audio('anechoic.wav')